In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import animation, rc
from IPython.display import HTML
matplotlib.rcParams['savefig.dpi'] = 100


In [ ]:
#dirname='/home/ychen/data/0529_L45_M10_b1_h1'
dirname='/home/ychen/data/0602_L45_M10_hydro'
#dirname='/home/ychen/data/0605_L45_M10_b1_hinf'
#dirname='/home/ychen/d9/FLASH4/MHD_Jet_3D/0522_nozzle_restart'
simtitle = dirname.split('/')[-1]

nozzledata = np.genfromtxt(os.path.join(dirname, 'nozzleVec.dat'))

endstep=6500

#endstep=-1
nstep = nozzledata[:,0]
ttnoz = nozzledata[:,1]
xx = nozzledata[:,2]
yy = nozzledata[:,3]
zz = nozzledata[:,4]


if False:
    nozzledata2 = np.genfromtxt(os.path.join(dirname, 'nozzleVec.dat.bak'))
    ttnoz2 = nozzledata2[:,1]
    xx2 = nozzledata2[:,2]
    yy2 = nozzledata2[:,3]
    zz2 = nozzledata2[:,4]
    thetas2 = np.arccos(zz2)/np.pi*180
    phis2 = np.arctan2(yy2, xx2)
    ax.plot(phis2, thetas2, c='r', lw=0.5, alpha=0.5)

thetas = np.arccos(zz)/np.pi*180
phis = np.arctan2(yy, xx)
fig = plt.figure(figsize=(6,6))
ax = plt.subplot(111, polar=True)

def plot_jiggle(endstep):
    ax.clear()
    ax.plot(phis[:endstep], thetas[:endstep], alpha=0.2)#, s=0.2, lw=0, alpha=0.2)
    
    ax.set_rlim(0,20)
    ax.set_title(simtitle,loc='left')
    ax.text(np.pi/4., 28, endstep, ha='right')
    #ax.plot(phis, thetas, alpha=0.5)
    

    
    tmyr=int(ttnoz[endstep]/3.15569E13)
    for i in range(1,tmyr+1):
        arg = np.argmin(abs(ttnoz-i*3.15569e13))
        ax.scatter(phis[arg], thetas[arg], marker='x', color='r', s=4)
        ax.annotate('%i Myr' % i, xy=(phis[arg], thetas[arg]), size=6, bbox=dict(facecolor='w', lw=0, alpha=0.6))
    #plt.show()
    plt.savefig('./jiggle_figures/%s_%06.0f.png' % (simtitle, endstep))
    #print './jiggle_figures/%s_%06.0f.png' % (simtitle, endstep)

for step in nstep[-1000:]:
    if (step % 200) == 0:
        plot_jiggle(int(step))


In [ ]:
%pdb

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( 0, 2))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

HTML(anim.to_html5_video())

# equivalent to rcParams['animation.html'] = 'html5'
rc('animation', html='html5')

